In [2]:
import geopandas as gpd
import pandas as pd
import math

counties = gpd.read_file(r"D:\Dropbox\work files\Policy impact on fire\county_cities_filtered(interaction).gpkg")
counties = counties.to_crs(epsg='4326')
counties.head()


,AD2004,NAME2004,SHENG,Count_,Sum_总人口,Sum_男,Sum_女,Sum_户籍人,Sum_少数民,Sum_非农人,...,Sum_出生人,Sum_死亡人,Sum_总受教,Sum_男受_1,Sum_女受_1,Sum_GDP201,常住减户籍,面积,人口密度,geometry
0,610116,长安区,陕西省,1,1083285.0,547062.0,536223.0,977513.0,6499.7100,240814.2555,...,100745.5050,41598.1440,11439489.60,5869975.26,5571356.97,274.410,105772,1598.214728,677.809421,"MULTIPOLYGON (((109.22460 33.87748, 109.21729 ..."
1,610125,户县,陕西省,1,556377.0,287369.0,269008.0,572282.0,834.5655,96420.1341,...,52076.8872,33215.7069,5324527.89,2873690.00,2453352.96,105.780,-15905,1276.696955,435.794100,"MULTIPOLYGON (((108.76184 33.95985, 108.76421 ..."
2,610124,周至县,陕西省,1,562768.0,291651.0,271117.0,662657.0,450.2144,57683.7200,...,69501.8480,33147.0352,4907336.96,2721103.83,2193336.53,53.160,-99889,2972.254795,189.340430,"MULTIPOLYGON (((108.44716 34.08542, 108.45443 ..."
3,611026,柞水县,陕西省,1,153398.0,82177.0,71221.0,156879.0,260.7766,21521.7394,...,14035.9170,11075.3356,1217980.12,693573.88,523474.35,29.974,-3481,2339.913722,65.557118,"MULTIPOLYGON (((109.59655 33.67462, 109.58577 ..."
4,610923,宁陕县,陕西省,1,70435.0,38345.0,32090.0,73106.0,2542.7035,12009.1675,...,5916.5400,4754.3625,567001.75,317113.15,249660.20,11.612,-2671,3640.915536,19.345409,"MULTIPOLYGON (((108.63221 33.15357, 108.62495 ..."


In [2]:
bounds = counties.bounds
lon1 = bounds.minx.min()
lon2 = bounds.maxx.max()
lat1 = bounds.miny.min()
lat2 = bounds.maxy.max()
lon1, lon2, lat1, lat2

(104.5699920654297, 122.69989776611328, 28.33074951171874, 44.160049438476555)

In [3]:

latStart = min(lat1, lat2);
lonStart = min(lon1, lon2);

#定义栅格大小(单位m)
accuracy = 5000;

#计算栅格的经纬度增加量大小▲Lon和▲Lat
deltaLon = accuracy * 360 / (2 * math.pi * 6371004 * math.cos((lat1 + lat2) * math.pi / 360));
deltaLat = accuracy * 360 / (2 * math.pi * 6371004);



In [4]:
from shapely.geometry import Point,Polygon,shape

#定义空的geopandas表
data = gpd.GeoDataFrame()

#定义空的list，后面循环一次就往里面加东西
LONCOL = []
LATCOL = []
geometry = []
HBLON1 = []
HBLAT1 = []

#计算总共要生成多少个栅格
#lon方向是lonsnum个栅格
lonsnum = int((lon2-lon1)/deltaLon)+1
#lat方向是latsnum个栅格
latsnum = int((lat2-lat1)/deltaLat)+1

for i in range(lonsnum):
    for j in range(latsnum):

        HBLON = i*deltaLon + (lonStart - deltaLon / 2)
        HBLAT = j*deltaLat + (latStart - deltaLat / 2)
        #把生成的数据都加入到前面定义的空list里面
        LONCOL.append(i)
        LATCOL.append(j)
        HBLON1.append(HBLON)
        HBLAT1.append(HBLAT)
        
        #生成栅格的Polygon形状
        #这里我们用周围的栅格推算三个顶点的位置，否则生成的栅格因为小数点取值的问题会出现小缝，无法完美覆盖
        HBLON_1 = (i+1)*deltaLon + (lonStart - deltaLon / 2)
        HBLAT_1 = (j+1)*deltaLat + (latStart - deltaLat / 2)
        geometry.append(Polygon([
        (HBLON-deltaLon/2,HBLAT-deltaLat/2),
        (HBLON_1-deltaLon/2,HBLAT-deltaLat/2),
        (HBLON_1-deltaLon/2,HBLAT_1-deltaLat/2),
        (HBLON-deltaLon/2,HBLAT_1-deltaLat/2)]))
        
#为geopandas文件的每一列赋值为刚刚的list
data['LONCOL'] = LONCOL
data['LATCOL'] = LATCOL
data['HBLON'] = HBLON1
data['HBLAT'] = HBLAT1
data['fire'] = 0
data['geometry'] = geometry


In [5]:
data.head()

,LONCOL,LATCOL,HBLON,HBLAT,fire,geometry
0,0,0,104.542115,28.308266,0,"POLYGON ((104.51424 28.28578, 104.56999 28.285..."
1,0,1,104.542115,28.353233,0,"POLYGON ((104.51424 28.33075, 104.56999 28.330..."
2,0,2,104.542115,28.398199,0,"POLYGON ((104.51424 28.37572, 104.56999 28.375..."
3,0,3,104.542115,28.443165,0,"POLYGON ((104.51424 28.42068, 104.56999 28.420..."
4,0,4,104.542115,28.488131,0,"POLYGON ((104.51424 28.46565, 104.56999 28.465..."


In [4]:
data_year = 2017
fire_df = pd.read_csv(r"D:\Dropbox\work files\Policy impact on fire\VIIRS_{}_clipped.csv".format(data_year))
fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473204 entries, 0 to 473203
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  473204 non-null  int64  
 1   id          473204 non-null  int64  
 2   date        473204 non-null  object 
 3   time        473204 non-null  int64  
 4   brightness  473204 non-null  float64
 5   confidence  473204 non-null  object 
 6   geometry    473204 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 25.3+ MB


In [5]:
geometry =gpd.GeoSeries.from_wkt(fire_df["geometry"])
fire_points = gpd.GeoDataFrame(
    fire_df[["id", "date", "time", "brightness", "confidence"]], geometry=geometry
)

473204

In [ ]:
#将栅格设置成wgs84坐标，保证坐标系的统一
data = data.set_geometry(geometry,crs =4326 )
grid = data[data.intersects(sh.unary_union)]
grid.plot()
